In [1]:
import numpy as np
import tensorrt
import tensorflow as tf
import keras
from keras import layers

2023-10-17 15:17:31.720415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

padded_inputs = tf.keras.utils.pad_sequences(raw_inputs, padding="post")
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [16]:
padded_inputs[0][0] = 0
print(padded_inputs)

[[   0  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [2]:
class MaskModel(keras.Model):
  def __init__(self):
    super().__init__()
    # self.mask = layers.Masking(mask_value=-999)
    self.conv1 = layers.Conv1D(32, 2, activation='relu')
    self.flatten = layers.Flatten()
    self.d1 = layers.Dense(128, activation='relu')
    self.d2 = layers.Dense(6)

  def call(self, x):
    # x = self.mask(x)
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MaskModel()

loss_object = tf.keras.losses.MeanSquaredError()

optimizer = tf.keras.optimizers.SGD()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_accuracy')


In [3]:
@tf.function
def train_step(seqs, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(seqs, training=True)
    loss = loss_object(labels, predictions)
    tf.print(loss)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [4]:
# x and y
x_train = np.array([[-999, 1, 1, 1, 1, -999]]).astype("float32")
y_train = np.array([[8, 3, 0, 8, 2, 1]]).astype("float32")
train_ds = tf.data.Dataset.from_tensors((x_train[..., tf.newaxis], y_train[..., tf.newaxis]))


In [5]:
mask = layers.Masking(mask_value=0)
conv1 = layers.Conv1D(1, 1)
flatten = layers.Flatten()
d1 = layers.Dense(128, activation='relu')
d2 = layers.Dense(6)

In [263]:
for seqs, labels in train_ds:
    seqs = mask(seqs)
    print(seqs._keras_mask)
    seqs = conv1(seqs)
    print(seqs)
    # seqs = flatten(seqs)
    # print(seqs.shape)
    # seqs = d1(seqs)
    # seqs = d2(seqs)
    # print(seqs)

tf.Tensor([[False False False False  True False]], shape=(1, 6), dtype=bool)
tf.Tensor(
[[[0.       ]
  [0.       ]
  [0.       ]
  [0.       ]
  [0.9546095]
  [0.       ]]], shape=(1, 6, 1), dtype=float32)


In [5]:
tf.keras.utils.set_random_seed(9)
EPOCHS = 1
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()

  for seqs, labels in train_ds:
    train_step(seqs, labels)

  # for test_images, test_labels in test_ds:
  #   test_step(test_images, test_labels)

  # print(
  #   f'Epoch {epoch + 1}, '
  #   f'Loss: {train_loss.result()}, '
  #   f'Accuracy: {train_accuracy.result() * 100}, '
  #   f'Test Loss: {test_loss.result()}, '
  #   f'Test Accuracy: {test_accuracy.result() * 100}'
  # )

7526.60352


In [7]:
train_loss.result()  

<tf.Tensor: shape=(), dtype=float32, numpy=23.57035>

In [62]:
np.mean([2.27850771, 2.51852608, 2.64956594])

2.4821999100000003

In [6]:
tf.keras.utils.set_random_seed(9)
a = keras.Input(shape=(30, 10))                                                          
b = layers.Masking(-999999999)(a)                                                         
b = layers.Conv1D(3, 3, padding='same')(a)                                                
b = layers.Flatten()(b)                                                                   
b = layers.Dense(1)(b)                                                                    
model = keras.Model(a, b)                                                                
                                                                                   
x = np.random.random((100, 30, 10))                                                                                                                                                            
y = np.random.random((100, 1))                                                     
x[30, 15, :] = -999999999                                                          
                                                                                   
model.compile('adam', 'mse')                                                       
model.fit(x, y)

4/4 [==============================] - 0s 2ms/step - loss: 3739404468224.0000


In [4]:
def change(a): 
    def w(b): 
        def x(c): 
            def y(d): 
                def z(e): 
                    print(a, b, c, d, e) 
                return z 
            return y 
        return x 
    return w 

t = change(10)(20)(30)(40)